In [1]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
from config_reg import cfg
from dataset import Dataset
from factory_reg import backbone
from random import randint
from tensorflow.python.framework import graph_util
import pandas as pd

class Regression(object):
    def __init__(self):
        self.Batch_Size = cfg.TRAIN.BATCHSIZE
        self.learn_rate_init = cfg.TRAIN.LEARN_RATE_INIT
        self.num_classes = cfg.TRAIN.REG_NUM
        self.trainset = Dataset(self.num_classes,'train')
        self.testset = Dataset(self.num_classes,'test')
        self.network = cfg.TRAIN.NETWORK
        self.train_txt = cfg.TRAIN.ANNO_PATH
        self.sess = tf.Session()
        self.model_type = cfg.TRAIN.NETWORK
        self.input_size = cfg.TRAIN.INPUTSIZE
        self.interval = cfg.TRAIN.SAVE
        #self.excel_path = './result.xlsx'
        #self.df = pd.DataFrame(columns=['Source', 'Quantity', 'Model_name', 'Epoch', 'Loss', 'Acc'])
        self.initial_weights = cfg.TRAIN.INITIAL_WEIGHT
        self.pretrain_mode = cfg.TRAIN.PRETRAIN_MODE
        self.epoch = cfg.TRAIN.EPOCH
        self.pretrain_model = cfg.TRAIN.BACKBONE_PRETRAIN
        self.moving_ave_decay = cfg.TRAIN.MOMENTUM
        self.save_path_ckpt = cfg.TRAIN.SAVE_PATH_CKPT
        self.save_path_pb = cfg.TRAIN.SAVE_PATH_PB
        self.quantity = len(open(self.train_txt, 'r').readlines())
        self.source = ''
        #self.get_src()
        self.keep_prob = tf.placeholder(dtype=tf.float32, name='dropout')
        #self.writer = pd.ExcelWriter(self.excel_path)
        self.moving_ave_decay = 0.995
        
       
        self.input_data = tf.placeholder(shape = [None, self.input_size, self.input_size, 3], dtype=tf.float32, name='input')
        self.input_labels = tf.placeholder(shape = [None, self.num_classes], dtype=tf.float32, name='label')
        self.input_scale = tf.placeholder(shape = [None, 1], dtype=tf.float32, name='scale')
        self.trainable = tf.placeholder(dtype=tf.bool, name='trainable')

        
        self.model = backbone(model=self.network, input_data=self.input_data, trainable=self.trainable, classes=self.num_classes, keep_prob=1.0, scale=self.input_scale)
        self.loss = self.model.compute_loss(labels=self.input_labels)
        self.net_var = tf.trainable_variables()
        self.varaibles_to_restore = [var for var in self.net_var if 'backbone' in var.name]
        
            
        #moving_ave = tf.train.ExponentialMovingAverage(self.moving_ave_decay).apply(tf.trainable_variables())
        self.global_step = tf.Variable(1.0, dtype=tf.float32, trainable=False, name='global_step')
        self.global_step_update = tf.assign_add(self.global_step, 1.0)
        self.optimizer =  tf.train.AdamOptimizer(self.learn_rate_init).minimize(self.loss, var_list=self.net_var)
        #self.optimizer = tf.train.MomentumOptimizer(learning_rate=self.learn_rate_init, momentum=0.8).minimize(self.loss)
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            with tf.control_dependencies([self.optimizer, self.global_step_update]):
                #with tf.control_dependencies([moving_ave]):
                self.train_op = tf.no_op()
        
            #self.optimizer = tf.train.MomentumOptimizer(learning_rate=self.learn_rate_init, momentum=0.8).minimize(self.loss)
        
        self.loader_backbone = tf.train.Saver(self.varaibles_to_restore)
        self.loader_whole = tf.train.Saver(tf.global_variables())
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=10)
        
        
#         tensor_name_list = [tensor.name for tensor in tf.get_default_graph().as_graph_def().node]
#         for tensor_name in tensor_name_list:
#             print(tensor_name,'\n')

        

    def train(self):
        #self.sess.run(tf.global_variables_initializer())
        if self.pretrain_mode == 'whole':
            try:
                print ('=>Restore weights from ' + self.initial_weights)
                self.loader_whole.restore(self.sess, self.initial_weights)
            except:
                print (self.initial_weights + 'does not exist!')
                print ('=>starts training from scratch ...')
        else:
            try:
                print ('=>Restore weights from ' + self.pretrain_model)
                self.loader_backbone.restore(self.sess, self.pretrain_model)
            except:
                print (self.pretrain_model + 'does not exist!')
                print ('=>starts training from scratch ...')
        
        min_loss_val = 20
        min_loss_train = 20
        i = 0
        for epoch in range(self.epoch):
            pabr = tqdm(self.trainset)
            train_epoch_loss, test_epoch_loss = [], []
            #train_epoch_acc, test_epoch_acc = [], []
            for train_data in pabr:
                _, train_step_loss = self.sess.run([self.train_op, self.loss], feed_dict={self.input_data: train_data[0],
                                                         self.input_labels: train_data[1],
                                                         self.input_scale: train_data[2],
                                                         self.trainable: True,
                                                         })
                
                train_epoch_loss.append(train_step_loss)
                #train_epoch_acc.append(train_step_acc)
                pabr.set_description("train loss: %.2f" %train_step_loss)
            
            for test_data in self.testset:
                test_step_loss = self.sess.run([self.loss],
                                                                feed_dict={self.input_data: test_data[0],
                                                                           self.input_labels: test_data[1],
                                                                           self.input_scale: test_data[2],
                                                                           self.trainable: False, 
                                                                           })

                test_epoch_loss.append(test_step_loss)
                #test_epoch_acc.append(test_step_acc)

            train_epoch_loss, test_epoch_loss = np.mean(train_epoch_loss), np.mean(test_epoch_loss)
            print ('Epoch: %2d Train loss: %.2f'
                   %(epoch, train_epoch_loss))

            print ('Test loss: %.2f '
                   % (test_epoch_loss))
            
            
            if epoch >= 5 and test_epoch_loss < min_loss_val and train_epoch_loss < min_loss_train:
                min_loss_val = test_epoch_loss
                min_loss_train = train_epoch_loss
                constant_graph = graph_util.convert_variables_to_constants(self.sess, self.sess.graph_def, ['fc_layer/op_to_store'])
                model_name = self.model_type+'_epoch=%d' %epoch
                ckpt_file = self.save_path_ckpt + model_name + '_test_loss=%.4f.ckpt' %test_epoch_loss
                #self.saver.save(self.sess, ckpt_file, global_step=epoch)
                with tf.gfile.FastGFile(self.save_path_pb + model_name+'.pb', mode='wb') as f:
                    f.write(constant_graph.SerializeToString())
#                 self.df.loc[i] = [randint(-1, 1) for _ in range(len(self.df.columns))]
#                 self.df.iloc[i,0] = self.source
#                 self.df.iloc[i,1] = str(self.quantity)
#                 self.df.iloc[i,2] = model_name
#                 self.df.iloc[i,3] = str(epoch)
#                 self.df.iloc[i,4] = str(test_epoch_loss)
#                 self.df.iloc[i,5] = str(test_epoch_acc)
#                 i += 1
                
                
#         self.df.to_excel(self.writer, 'Sheet1')
#         self.writer.save()
                

    def main(self):
        self.sess.run(tf.global_variables_initializer())
        self.train()
        
if __name__ == '__main__':
    Regression().main()





Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
=>Restore weights from /hdd/sd5/tlc/pretrain_model/darknet53/yolov3_test_loss=136.5372.ckpt-80
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /hdd/sd5/tlc/pretrain_model/darknet53/yolov3_test_loss=136.5372.ckpt-80


  0%|          | 0/494 [00:00<?, ?it/s]

/hdd/sd5/tlc/pretrain_model/darknet53/yolov3_test_loss=136.5372.ckpt-80does not exist!
=>starts training from scratch ...


train loss: 3.28:   0%|          | 2/494 [00:00<00:47, 10.34it/s]

Epoch:  0 Train loss: 4.10
Test loss: 41.48 


...

Epoch: 94 Train loss: 0.39
Test loss: 2.37 


train loss: 0.80:   0%|          | 2/494 [00:00<00:41, 11.87it/s]

Epoch: 95 Train loss: 0.39
Test loss: 2.41 


train loss: 0.30:  19%|█▊        | 92/494 [00:07<00:28, 14.05it/s]

KeyboardInterrupt: 